<a href="https://colab.research.google.com/github/HelmchenLabSoftware/Cascade/blob/adrian_dev/Demo%20scripts/Visualize_ground_truth_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASCADE

## Visualization of Ground Truth Datasets


TODO: describe a bit calcium recording with cell attached spike recording, different datasets, varying indicators, frame rates, animals...

TODO: add refererence to the people who recorded/provided these datasets

TODO: Change link in badge above and the git clone command to the master branch as soon as it is merged.




##1. Load data and import/download required code

In [1]:
#@markdown Execute the code by clicking on the **play** symbol on the left of each cell.

#@markdown If executed within a colaboratory notebook, this code downloads the repository to the current environment. This can take a couple of seconds.

#@markdown After downloading the repository, the ground truth dataset with simultaneously recorded fluorescence and spiking activity are loaded.

#@markdown *Note: You can check the code underlying each cell by double-clicking on it.*

import os

# If in Colab and not yet downloaded, download GitHub repository
if os.getcwd() == '/content': 
    if not os.path.exists('/content/Cascade'):
      !git clone -b adrian_dev https://github.com/HelmchenLabSoftware/Cascade
    os.chdir('Cascade')
    
# If executed as jupyter notebook on own computer, change to parent directory for imports
if os.path.basename( os.getcwd() ) == 'Demo scripts':
    %cd ..
    print('New working directory:', os.getcwd() )

# set environment variables depending on colab or jupyter environment
if os.getcwd() == '/content/Cascade':
  ENV = 'colab'
else: # assume jupyter
  ENV = 'jupyter'
  %matplotlib notebook   # does not work in colab unfortunately

# Imports:
import matplotlib.pyplot as plt
import numpy as np

from cascade2p import utils
import ipywidgets as widgets

## Load datasets
datasets = utils.load_all_ground_truth( ground_truth_folder='Ground_truth')
dataset_names = list( datasets.keys() )

print('\nSuccessfully loaded ground truth data.')

Cloning into 'Cascade'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 1444 (delta 101), reused 99 (delta 49), pack-reused 1266
Receiving objects: 100% (1444/1444), 550.39 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (383/383), done.
Checking out files: 100% (428/428), done.

Successfully loaded ground truth data.


##2. Explore the ground truth data

In [2]:
#@markdown Click the play button on the left, then select the dataset in the dropdown menu and the recording with the slider below.

#@markdown You can zoom into the plot with the two sliders for offset and zoom.

#@markdown If you run this in a jupyter notebook, you can use the magnifying glass button to draw rectangles with the left mouse button to zoom in, and the right mouse button to zoom out.

def plot_neuron(Dataset, Recording, Zoom=1, Offset=0):
    """Plot the fluorescence and spikes for a single recording of the dataset
    
    Parameters
    -------
    Dataset: str
        Name of the dataset, which is identical to the folder name in the Ground_truth folder
    Recording: int
        Number of the recording in this dataset, pooled across all files in this dataset
    """

    # correct for different plot sizes in colab and jupyter
    if ENV=='colab':   plt.figure(figsize=(20,8))
    if ENV=='jupyter': plt.figure(figsize=(10,8))
    
    if Recording >= len(datasets[Dataset]):
        print('Value {} is too large. Only {} recordings in this dataset.'.format(Recording, len(datasets[Dataset])))
        return
    
    data = datasets[Dataset][Recording]
    # plt.cla()
    plt.plot( data['t'], data['dff'], lw=0.5)
    
    max_dff = np.max( data['dff'])
    min_dff = np.min( data['dff'])
    
    plt.eventplot( data['spikes'], lineoffsets=min_dff - max_dff/20,
                  linelengths=max_dff/20, color='k', lw=1.5)
    
    # TODO: add number of APs if they are very close to each other
    
    plt.xlabel('Time [s]')
    plt.ylabel('Relative Fluorescence Change [Df/f]')
    plt.title('Dataset: {}   Recording: {}   Frame rate: {:.1f}Hz'.format(Dataset, Recording, data['frame_rate']))
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Adjust zoom and offset
    if (Zoom != 1) or (Offset != 0):
      t_min = Offset
      t_max = (data['t'][-1] / Zoom) + Offset
      plt.xlim( (t_min, t_max) )
    

widgets.interact(plot_neuron, Dataset=dataset_names,
                            Recording=widgets.IntSlider(min=0, max=100, step=1, value=0, continuous_update=False),
                            Zoom=widgets.IntSlider(min=1, max=50, step=1, value=1, continuous_update=False),
                            Offset=widgets.IntSlider(min=0, max=100, step=1, value=0, continuous_update=False)            
                );


interactive(children=(Dropdown(description='Dataset', options=('X-DS09-GCaMP6f-m-V1', 'DS11-GCaMP6s-m-V1-neuro…